## Introduction
Because Pittsburgh is always called a food city, we decided to base our metric on density of food per Allegheny county zip code. By "food", we mean places that residents can buy food, like supermarkets, convenience stores, restaurants, and farmers markets. Combined, these places will give us a good idea of how much food there actually is per zip code. Some other options we explored included sorting dog names by neighborhood, health quality per zip code, and food quality per zip code.

## Metric
Our data sets are the following: 
* [Allegheny County Supermarkets and Convenience Stores](https://data.wprdc.org/dataset/allegheny-county-supermarkets-convenience-stores/resource/626357fa-c95d-465f-9a02-3121655b2b78)
* [Allegheny County Farmers Markets](https://data.wprdc.org/dataset/allegheny-county-farmers-markets-locations)
* [Allegheny County Restaurant Food Facility Inspections and Locations](https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations)

Our metric is the amount of food facilities per zip code. We chose this specifically because all three of these data sets have a zip code listing. This makes it easier to combine all three of these datasets into one, as they all have a common column.  

In [2]:
import pandas as pd
import numpy as np


Convenience store stuff

In [22]:
conveniencemarkets = pd.read_csv("data-conveniencesupermarkets.csv")
conveniencemarkets.drop(['Client ID', 'Legal Name', 'Start Date', 'Street #', 'Street Name', 'State', 'Lat', 'Lon', 'Accuracy', 'Category'], axis=1, inplace=True)
conveniencemarkets.sort_values('Zip').head(10)


,Name,Zip
223,DiLeo's Deli,15014.0
31,CR Petroleum,15014.0
126,7 Eleven #36135,15017.0
520,Aldi #81,15017.0
290,Getgo #3641,15017.0
549,Giant Eagle #24,15017.0
197,Co Go's #322,15017.0
425,South Fayette Sunoco,15017.0
607,Giant Eagle #641,15017.0
297,Gulf Gas Station - Bridgeville,15017.0


Restaurant stuff 

In [23]:
restaurants = pd.read_csv('GeocodedFoodFacilities.csv')
restaurants.drop_duplicates(subset = ['facility_name','num'], keep = 'first', inplace = True)
restaurants.drop(['id', 'num', 'street', 'city', 'state', 'municipal', 'category_cd', 'description', 'fdo', 'bus_st_date', 'seat_count', 'noroom', 'sq_feet', 'status', 'placard_st', 'x', 'y', 'p_code', 'address'], axis=1, inplace=True)
restaurants.sort_values('zip').head(10)

,facility_name,zip
8940,Coco Bongos,14530
11251,Patti's Pastries (XTK-7969) MFF3,15001
11867,The Humble Cookie Stand (XKZ-3096) MFF3,15001
8328,Palms Brazilian Steakhouse - Mobile,15001
3772,Pizza Joe's,15003
557,Fair Oaks Lounge,15003
2993,Berkey's Tavern,15003
5853,The Halfway House,15006
1377,Bairdford United Methodist Church,15006
7865,St Victor Parish Activity Hall Kitchen,15006
